In [50]:
#######   DRAWING ###########
from __future__ import print_function
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
from IPython.display import display, HTML
import itertools
############# NUMPY ###############
import numpy as np
from numpy import array
from numpy import argmax
from numpy import asarray
from numpy import load
from numpy import save
from numpy import savez_compressed
from numpy.random import seed
seed(1)
############## SKLEARN ################
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
########### OTHER ##########
import pandas as pd
import math
import copy
import time
from PIL import Image
import png
import numpy as geek

In [51]:
########################### CONSTANTS ##########################
NUM_CLASSES = 6
MY_ACTIVITIES = ['Downstairs', 'Jogging', 'Sitting', 'Standing', 'Upstairs', 'Walking']
MY_LABEL = 'ActivityEncoded'
pd.options.display.float_format = '{:.1f}'.format
sns.set() # Default seaborn look and feel
plt.style.use('ggplot')
# The number of steps within one time segment
MY_WINDOW_SIZE = 40 # window size
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
MY_STEP_DISTANCE = 40
MY_CHANNEL_SIZE = 3
# Hyper-parameters
MY_BATCH_SIZE = 400
MY_EPOCHS = 50
w_area = 20
h_area = 20
num_area_w = 6
num_area_h = 6
dict_color = {'b': (0, 0, 0),
              'w': (255, 255, 255)}
              
dictionary = {'x': 0, 'y': 1, 'z': 2}

'''
  '5': 'Walking',
  '4': 'Upstairs',
  '3': 'Standing',
  '2': 'Sitting',
  '1': 'Jogging',
  '0': 'Downstairs',
  '''

"\n  '5': 'Walking',\n  '4': 'Upstairs',\n  '3': 'Standing',\n  '2': 'Sitting',\n  '1': 'Jogging',\n  '0': 'Downstairs',\n  "

In [52]:
######################## HELPER FUNCTIONS ######################

# This function read the data in the format of "WISDM" dataset. 
# Both with and without semicolomn ending
def read_data(file_path):
    column_names = ['user-id',
                    'activity',
                    'timestamp',
                    'x-axis',
                    'y-axis',
                    'z-axis']
    df = pd.read_csv(file_path,
                     header=None,
                     names=column_names)
    if str(df['z-axis'][0]).endswith(';'):
      # Last column has a ";" character which must be removed ...
      df['z-axis'].replace(regex=True,
        inplace=True,
        to_replace=r';',
        value=r'')
      # ... and then this column must be transformed to float explicitly
      df['z-axis'] = df['z-axis'].apply(convert_to_float)
      # This is very important otherwise the model will not fit and loss
      # will show up as NAN
    df.dropna(axis=0, how='any', inplace=True)

    return df

def convert_to_float(x):

    try:
        return np.float(x)
    except:
        return np.nan
 
def show_basic_dataframe_info(dataframe):

    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

def plot_activity(activity, data):

    fig, (ax0, ax1, ax2) = plt.subplots(nrows=3,
         figsize=(15, 10),
         sharex=True)
    plot_axis(ax0, data['timestamp'], data['x-axis'], 'X-Axis')
    plot_axis(ax1, data['timestamp'], data['y-axis'], 'Y-Axis')
    plot_axis(ax2, data['timestamp'], data['z-axis'], 'Z-Axis')
    plt.subplots_adjust(hspace=0.2)
    fig.suptitle(activity)
    plt.subplots_adjust(top=0.90)
    plt.show()

def plot_axis(ax, x, y, title):

    ax.plot(x, y, 'r')
    ax.set_title(title)
    ax.xaxis.set_visible(False)
    ax.set_ylim([min(y) - np.std(y), max(y) + np.std(y)])
    ax.set_xlim([min(x), max(x)])
    ax.grid(True)

def create_segments_and_labels(df, time_steps, step, label_name):

    # x, y, z acceleration as features
    N_FEATURES = 3
    # Number of steps to advance in each iteration (for me, it should always
    # be equal to the time_steps in order to have no overlap between segments)
    # step = time_steps
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
        xs = df['x-axis'].values[i: i + time_steps]
        ys = df['y-axis'].values[i: i + time_steps]
        zs = df['z-axis'].values[i: i + time_steps]
        # Retrieve the most often used label in this segment
        label = stats.mode(df[label_name][i: i + time_steps])[0][0]
        segments.append([xs, ys, zs])
        labels.append(label)

    # Bring the segments into a better shape
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, N_FEATURES)
    labels = np.asarray(labels)

    return reshaped_segments, labels

def standardize(train, test):
	""" Standardize data """

	# Standardize train and test
	X_train = (train - np.mean(train, axis=0)[None,:,:]) / np.std(train, axis=0)[None,:,:]
	X_test = (test - np.mean(test, axis=0)[None,:,:]) / np.std(test, axis=0)[None,:,:]    
	return X_train, X_test

def feature_normalize(dataset):
  mu = np.mean(dataset,axis = 0)
  sigma = np.std(dataset,axis = 0)
  return (dataset - mu)/sigma

def my_train_test_splitter(dataset, test_size):
  gb = dataset.groupby('activity')
  df_downstairs, df_jogging, df_sitting, df_standing, df_upstairs, df_walking = [gb.get_group(x) for x in gb.groups]

  S_train_raw_downstairs, S_test_raw_downstairs, L_train_raw_downstairs, L_test_raw_downstairs = train_test_split(df_downstairs.iloc[:, 3:6].values, 
                                                                                                                  df_downstairs.iloc[:, 6:7].values, 
                                                                                                                  test_size = test_size, 
                                                                                                                  random_state = 0)
  S_train_raw_jogging, S_test_raw_jogging, L_train_raw_jogging, L_test_raw_jogging  = train_test_split(df_jogging.iloc[:, 3:6].values, 
                                                                                                       df_jogging.iloc[:, 6:7].values, 
                                                                                                       test_size = test_size, 
                                                                                                       random_state = 0)
  S_train_raw_sitting, S_test_raw_sitting, L_train_raw_sitting, L_test_raw_sitting = train_test_split(df_sitting.iloc[:, 3:6].values, 
                                                                                                      df_sitting.iloc[:, 6:7].values, 
                                                                                                      test_size = test_size, 
                                                                                                      random_state = 0)
  S_train_raw_standing, S_test_raw_standing, L_train_raw_standing, L_test_raw_standing = train_test_split(df_standing.iloc[:, 3:6].values, 
                                                                                                          df_standing.iloc[:, 6:7].values, 
                                                                                                          test_size = test_size, 
                                                                                                          random_state = 0)
  S_train_raw_upstairs, S_test_raw_upstairs, L_train_raw_upstairs, L_test_raw_upstairs = train_test_split(df_upstairs.iloc[:, 3:6].values, 
                                                                                                          df_upstairs.iloc[:, 6:7].values, 
                                                                                                          test_size = test_size, 
                                                                                                          random_state = 0)
  S_train_raw_walking, S_test_raw_walking, L_train_raw_walking, L_test_raw_walking = train_test_split(df_walking.iloc[:, 3:6].values, 
                                                                                                      df_walking.iloc[:, 6:7].values, 
                                                                                                      test_size = test_size, 
                                                                                                      random_state = 0)
   
  S_train = np.concatenate((S_train_raw_walking, 
                            S_train_raw_upstairs, 
                            S_train_raw_standing, 
                            S_train_raw_sitting, 
                            S_train_raw_jogging, 
                            S_train_raw_downstairs), 
                           axis=0)
  S_test = np.concatenate((S_test_raw_walking, 
                           S_test_raw_upstairs, 
                           S_test_raw_standing, 
                           S_test_raw_sitting, 
                           S_test_raw_jogging, 
                           S_test_raw_downstairs), 
                          axis=0)
  L_train = np.concatenate((L_train_raw_walking, 
                            L_train_raw_upstairs, 
                            L_train_raw_standing, 
                            L_train_raw_sitting, 
                            L_train_raw_jogging, 
                            L_train_raw_downstairs), 
                           axis=0)
  L_test = np.concatenate((L_test_raw_walking, 
                           L_test_raw_upstairs, 
                           L_test_raw_standing, 
                           L_test_raw_sitting, 
                           L_test_raw_jogging, 
                           L_test_raw_downstairs), 
                          axis=0)      

  return S_train, S_test, L_train, L_test

def plot_activity(activity, data):

    fig, (ax0, ax1, ax2) = plt.subplots(nrows=3,
         figsize=(15, 10),
         sharex=True)
    plot_axis(ax0, data['timestamp'], data['x-axis'], 'X-Axis')
    plot_axis(ax1, data['timestamp'], data['y-axis'], 'Y-Axis')
    plot_axis(ax2, data['timestamp'], data['z-axis'], 'Z-Axis')
    plt.subplots_adjust(hspace=0.2)
    fig.suptitle(activity)
    plt.subplots_adjust(top=0.90)
    plt.show()

def plot_axis(ax, x, y, title):

    ax.plot(x, y, 'r')
    ax.set_title(title)
    ax.xaxis.set_visible(False)
    ax.set_ylim([min(y) - np.std(y), max(y) + np.std(y)])
    ax.set_xlim([min(x), max(x)])
    ax.grid(True)

In [53]:
###### MOUNT GOOGLE DRIVE AS A FOLDER ###### 
#drive.mount('/content/drive')
#data_path = '/content/drive/My Drive/Colab Notebooks/dataset/'
data_path = os.path.abspath("Gen_WISDM_D4.ipynb")
print(data_path)

/Users/mesrbn/Desktop/Data_Gen_Codes/WISDM/Gen_WISDM_D4.ipynb


In [54]:
## READ DATA AND DROP NA ########
#dataset = read_data(data_path +'WISDM_ar_v1.1_raw.txt')
dataset = read_data('/Users/mesrbn/Desktop/Data_Gen_Codes/WISDM/wisdm_raw_data.txt')
df = dataset.loc[:, ['user-id','activity','timestamp','x-axis','y-axis', 'z-axis']]
dropped_na_dataset = df.dropna(axis = 0, how ='any')
# comparing sizes of data frames 
print("Old data frame length:", len(dataset), "\nNew data frame length:",  
       len(dropped_na_dataset), "\nNumber of rows with at least 1 NA value: ", 
       (len(dataset)-len(dropped_na_dataset))) 
df = dropped_na_dataset

/var/folders/w3/skl35jlj5jzc2cvk9z8nxw640000gn/T/ipykernel_90438/212540442.py:32: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.float(x)


Old data frame length: 1098203 
New data frame length: 1098203 
Number of rows with at least 1 NA value:  0


In [55]:
# Describe the data
show_basic_dataframe_info(df)
df.head(2)


Number of columns in the dataframe: 6
Number of rows in the dataframe: 1098203



,user-id,activity,timestamp,x-axis,y-axis,z-axis
0,33,Jogging,49105962326000,-0.7,12.7,0.5
1,33,Jogging,49106062271000,5.0,11.3,1.0


In [56]:
user_groups = df.groupby('user-id')
user_33 = user_groups.get_group(33)
user_33_walking = user_33.groupby('activity').get_group('Walking')
user_33_walking

,user-id,activity,timestamp,x-axis,y-axis,z-axis
597,33,Walking,49394992294000,0.8,8.0,2.8
598,33,Walking,49395102310000,1.1,8.6,3.8
599,33,Walking,49395202316000,-0.5,16.7,1.3
600,33,Walking,49395302292000,4.8,10.8,-1.2
601,33,Walking,49395412338000,-0.0,9.2,-0.7
...,...,...,...,...,...,...
66051,33,Walking,11318802348000,-3.4,1.3,6.6
66052,33,Walking,11318852519000,-6.8,2.3,8.0
66053,33,Walking,11318902568000,-6.6,2.8,9.7
66054,33,Walking,11318962322000,-7.8,1.7,6.6


In [57]:
user_33_walking = user_33_walking.iloc[:, [3, 4, 5]]
print(user_33_walking.shape)

(14898, 3)


In [58]:
activity_groups = df.groupby('activity')
activity_walking = activity_groups.get_group('Walking').iloc[:, [3, 4, 5]]
activity_jogging = activity_groups.get_group('Jogging').iloc[:, [3, 4, 5]]
activity_sitting = activity_groups.get_group('Sitting').iloc[:, [3, 4, 5]]
activity_standing = activity_groups.get_group('Standing').iloc[:, [3, 4, 5]]
activity_upstairs = activity_groups.get_group('Upstairs').iloc[:, [3, 4, 5]]
activity_downstairs = activity_groups.get_group('Downstairs').iloc[:, [3, 4, 5]]


In [59]:
################ HELPER FUNCTIONS #####################
def data_segmentation(data, channel_size, number_of_rows, window_size):
  #print(data)
  number_of_windows = int(math.floor(float(number_of_rows)/window_size))
  print("number of windows = ", number_of_windows)
  extra_rows = (number_of_windows * window_size) - number_of_rows
  print("number of extra rows = ", extra_rows)
  if extra_rows != 0:
    data = data[:extra_rows] #drop extra rows
  data = data.values
  segmented_data = np.reshape(data, (number_of_windows, window_size, channel_size))
  
  print("Proof of right segmentation:")
  print("(window size) * (number of windows) + extra rows = ", (number_of_windows * window_size) - extra_rows)  
  print("Raw data size before segmentation was:", len(data) - extra_rows)
  return segmented_data


  

In [60]:
segmented_data = data_segmentation(data=user_33_walking, channel_size=MY_CHANNEL_SIZE, number_of_rows=user_33_walking.shape[0], window_size=MY_WINDOW_SIZE)
segmented_data_walking = data_segmentation(data=activity_walking, channel_size=MY_CHANNEL_SIZE, number_of_rows=activity_walking.shape[0], window_size=MY_WINDOW_SIZE)
segmented_data_jogging = data_segmentation(data=activity_jogging, channel_size=MY_CHANNEL_SIZE, number_of_rows=activity_jogging.shape[0], window_size=MY_WINDOW_SIZE)
segmented_data_sitting = data_segmentation(data=activity_sitting, channel_size=MY_CHANNEL_SIZE, number_of_rows=activity_sitting.shape[0], window_size=MY_WINDOW_SIZE)
segmented_data_standing = data_segmentation(data=activity_standing, channel_size=MY_CHANNEL_SIZE, number_of_rows=activity_standing.shape[0], window_size=MY_WINDOW_SIZE)
segmented_data_upstairs = data_segmentation(data=activity_upstairs, channel_size=MY_CHANNEL_SIZE, number_of_rows=activity_upstairs.shape[0], window_size=MY_WINDOW_SIZE)
segmented_data_downstairs = data_segmentation(data=activity_downstairs, channel_size=MY_CHANNEL_SIZE, number_of_rows=activity_downstairs.shape[0], window_size=MY_WINDOW_SIZE)
print(segmented_data.shape)

number of windows =  372
number of extra rows =  -18
Proof of right segmentation:
(window size) * (number of windows) + extra rows =  14898
Raw data size before segmentation was: 14898
number of windows =  10609
number of extra rows =  -37
Proof of right segmentation:
(window size) * (number of windows) + extra rows =  424397
Raw data size before segmentation was: 424397
number of windows =  8554
number of extra rows =  -16
Proof of right segmentation:
(window size) * (number of windows) + extra rows =  342176
Raw data size before segmentation was: 342176
number of windows =  1498
number of extra rows =  -19
Proof of right segmentation:
(window size) * (number of windows) + extra rows =  59939
Raw data size before segmentation was: 59939
number of windows =  1209
number of extra rows =  -35
Proof of right segmentation:
(window size) * (number of windows) + extra rows =  48395
Raw data size before segmentation was: 48395
number of windows =  3071
number of extra rows =  -29
Proof of rig

In [61]:
print((segmented_data_walking.shape))

(10609, 40, 3)


In [62]:
first_segment = segmented_data[0]
first_segment = np.round(first_segment, 3) 
print(first_segment.shape)

(40, 3)


In [63]:
x_first_segment = [i[0] for i in first_segment]
print(x_first_segment)

[0.844, 1.117, -0.504, 4.794, -0.041, 2.493, 0.531, 1.757, 2.983, -0.804, 0.191, -0.341, 0.463, 0.082, -0.763, -1.607, 1.457, 1.035, 2.874, -3.95, 4.372, -1.798, 1.92, -4.44, 1.457, 1.92, 1.88, 2.765, -1.144, 4.141, -1.335, 1.335, -4.181, 1.73, 0.15, 2.983, 2.833, -4.181, 5.285, -2.138]


In [64]:
def cell_speed_long(data):
  result = data[-1] - data[0]
  return round(result/len(data), 3)

def cell_reach_maximum(data):
  return max(data)

def cell_reach_difference(data):
  result = max(data) - min(data)
  return result

def cal_pre_curr_post(data, seg_number, axis):
  #pre_pre_seg_data = []
  #post_post_seg_data = []
  current_seg_data = np.round([i[dictionary.get(axis, 'x')] for i in data[seg_number]], 3)
  if seg_number == 0:
    pre_seg_data = np.zeros(MY_WINDOW_SIZE)
    pre_pre_seg_data = np.zeros(MY_WINDOW_SIZE)
  elif seg_number == 1:
    pre_seg_data = np.round([i[dictionary.get(axis, 'x')] for i in data[seg_number-1]], 3)
    pre_pre_seg_data = np.zeros(MY_WINDOW_SIZE)
  else:
    pre_seg_data = np.round([i[dictionary.get(axis, 'x')] for i in data[seg_number-1]], 3)
    pre_pre_seg_data = np.round([i[dictionary.get(axis, 'x')] for i in data[seg_number - 2]], 3)

  if seg_number == (data.shape[0]-1):
    post_seg_data = np.zeros(MY_WINDOW_SIZE)
    post_post_seg_data = np.zeros(MY_WINDOW_SIZE)
  elif seg_number == (data.shape[0]-2):
    post_seg_data = np.round([i[dictionary.get(axis, 'x')] for i in data[seg_number+1]], 3)
    post_post_seg_data = np.zeros(MY_WINDOW_SIZE)
  else:
    post_seg_data = np.round([i[dictionary.get(axis, 'x')] for i in data[seg_number+1]], 3)
    post_post_seg_data = np.round([i[dictionary.get(axis, 'x')] for i in data[seg_number + 2]], 3)
  return pre_pre_seg_data, pre_seg_data, current_seg_data, post_seg_data, post_post_seg_data

def cal_it_lg_pre_curr_post(data, seg_number, axis):
  pre_pre_seg_data, pre_seg_data, current_seg_data, post_seg_data, post_post_seg_data = cal_pre_curr_post(data, seg_number, axis)
  iter_pre, lg_pre = cal_it_lg_values(np.concatenate((pre_pre_seg_data, pre_seg_data, current_seg_data), axis=None))  ## 2w    w    2w
  iter_curr, lg_curr = cal_it_lg_values(current_seg_data)
  iter_post, lg_post = cal_it_lg_values(np.concatenate((current_seg_data , post_seg_data, post_seg_data), axis=None)) ## 2w    w    2w
  return [iter_pre, iter_curr, iter_post], [lg_pre, lg_curr, lg_post]

def cal_reach_maximum(data, seg_number, axis):
  pre_pre_seg_data, pre_seg_data, current_seg_data, post_seg_data, post_post_seg_data = cal_pre_curr_post(data, seg_number, axis)
  result = [cell_reach_maximum(np.concatenate((pre_pre_seg_data, pre_seg_data, current_seg_data), axis=None)), cell_reach_maximum(current_seg_data), cell_reach_maximum(np.concatenate((current_seg_data, post_seg_data, post_post_seg_data), axis=None))]
  return [round(x, 2) for x in result]

def cal_reach_difference(data, seg_number, axis):
  pre_pre_seg_data, pre_seg_data, current_seg_data, post_seg_data, post_post_seg_data = cal_pre_curr_post(data, seg_number, axis)
  result = [cell_reach_difference(np.concatenate((pre_pre_seg_data, pre_seg_data, current_seg_data), axis=None)), cell_reach_difference(current_seg_data), cell_reach_difference(np.concatenate((current_seg_data, post_seg_data, post_post_seg_data), axis=None))]
  return [round(x, 2) for x in result]

def create_matrix(data, seg_number):
  it_x, lg_x = cal_it_lg_pre_curr_post(data, seg_number, 'x')
  it_y, lg_y = cal_it_lg_pre_curr_post(data, seg_number, 'y')
  it_z, lg_z = cal_it_lg_pre_curr_post(data, seg_number, 'z')
  result = np.array([it_x + lg_x])
  result = np.concatenate((result, [it_y + lg_y]), axis=0)
  result = np.concatenate((result, [it_z + lg_z]), axis=0)
  result = np.concatenate((result, [cal_reach_maximum(data, seg_number, 'x') + cal_reach_difference(data, seg_number, 'x')]), axis=0)
  result = np.concatenate((result, [cal_reach_maximum(data, seg_number, 'y') + cal_reach_difference(data, seg_number, 'y')]), axis=0)
  result = np.concatenate((result, [cal_reach_maximum(data, seg_number, 'z') + cal_reach_difference(data, seg_number, 'z')]), axis=0)
  return result

def cal_it_lg_values(test):
  it_list = []
  lg_list = []
  for i in range(0, len(test)):
    if (i + 2 < len(test)):
      block = [test[i], test[i + 1], test[i + 2]]
      num1 = block[1] - block[0]
      num2 = block[2] - block[1]
      if (abs(num1 + num2) > max(abs(num1), abs(num2))):
        lg_list.append(block)
      else:
        it_list.append(block)
      block.append(i)


  iter = True
  while iter:
    iter = False
    for idx, val in enumerate(it_list):
      if (idx != 0):
          if ((val[-1] - it_list[idx - 1][-1]) == 1):
            it_list[idx - 1][-1] = val[-2]
            it_list[idx - 1].append(val[-1])
            iter = True
            del it_list[idx]
            break


  iter = True
  count = 0
  while iter:
    iter = False
    for idx, val in enumerate(lg_list):
      if (idx != 0):
        if ((val[-1] - lg_list[idx - 1][-1]) == 1):
          lg_list[idx - 1][-1] = val[-2]
          lg_list[idx - 1].append(val[-1])
          iter = True
          del lg_list[idx]
          break

  for val in it_list:
    del val[-1]
  for val in lg_list:
    del val[-1]

  speed_long_result = 0
  for list_of_val in lg_list:
    result = 0
    result = result + (abs(list_of_val[-1] - list_of_val[0]))
    result = result / len(list_of_val)
    speed_long_result = round(speed_long_result + result, 3)

  speed_iter_result = 0
  for list_of_val in it_list:
    result = 0
    for idx in range(0, len(list_of_val)):
      if(idx+1 < len(list_of_val)):
        result = result + (abs(list_of_val[idx + 1] - list_of_val[idx]))

    result = result / len(list_of_val)
    speed_iter_result = round(speed_iter_result + result, 3)

  return speed_iter_result, speed_long_result

#https://www.quora.com/How-do-you-loop-through-the-elements-of-an-NxM-matrix-in-spiral-order
#http://javabypatel.blogspot.com/2016/11/print-matrix-in-spiral-order.html
#http://javabypatel.blogspot.com/2016/11/print-matrix-in-spiral-order-recursion.html
#https://www.geeksforgeeks.org/print-a-given-matrix-in-spiral-form/

def clockwise_inwards(height, width, number_of_black_pixel, rgb_list_selected, rgb_list_unselected):  
  rgb_list_unselected_area = [ -x for x in rgb_list_unselected]
  #img = []
  pattern = []
  pixel_order_list = []
  for y in range(width):
    my_row = []
    #row = ()
    for x in range(height):
      my_row.append([255, 255, 255])
      #row = row + (255, 255, 255)
    #img.append(row)
    pattern.append(my_row)
  #print(img)  

  rowStart = 0
  rowLength = height-1
  colStart = 0
  colLength  = width-1
    
  while(rowStart <= rowLength and colStart <= colLength):
    for i in range(rowStart, colLength+1, 1):
      #print(matrix[rowStart][i] , " ")
      pixel_order_list.append((rowStart, i))
    for j in range(rowStart+1, rowLength+1):
      #print(matrix[j][colLength] , " ")
      pixel_order_list.append((j, colLength))
    if rowStart+1 <= rowLength:
      for k in range(colLength-1, colStart-1, -1):
        #print(matrix[rowLength][k] , " ")
        pixel_order_list.append((rowLength, k))
    if colStart+1 <= colLength:
      for k in range(rowLength-1, rowStart, -1):
        #print(matrix[k][colStart] , " ")
        pixel_order_list.append((k, colStart))
    rowStart +=1
    rowLength -=1
    colStart +=1
    colLength -=1

  for i in range(0, number_of_black_pixel):
    row = pixel_order_list[i][0]
    k = pixel_order_list[i][1]
    k+=1
    pattern[row] = pattern[row][:k-1] + [[sum(i) for i in zip([0, 0, 0], rgb_list_selected)] ] + pattern[row][k:]

  for i in range(number_of_black_pixel, len(pixel_order_list)):
    row = pixel_order_list[i][0]
    k = pixel_order_list[i][1]
    k+=1
    pattern[row] = pattern[row][:k-1] + [[sum(i) for i in zip([255, 255, 255], rgb_list_unselected_area)] ] + pattern[row][k:]

  return pattern


def counter_clockwise_outwards(height, width, number_of_black_pixel, rgb_list_selected, rgb_list_unselected):  
  rgb_list_unselected_area = [ -x for x in rgb_list_unselected]
  #img = []
  pattern = []
  pixel_order_list = []
  for y in range(width):
    my_row = []
    #row = ()
    for x in range(height):
      my_row.append([255, 255, 255])
      #row = row + (255, 255, 255)
    #img.append(row)
    pattern.append(my_row)
  #print(img)  

  rowStart = 0
  rowLength = height-1
  colStart = 0
  colLength  = width-1
    
  while(rowStart <= rowLength and colStart <= colLength):
    for i in range(rowStart, colLength+1, 1):
      #print(matrix[rowStart][i] , " ")
      pixel_order_list.append((rowStart, i))
    for j in range(rowStart+1, rowLength+1):
      #print(matrix[j][colLength] , " ")
      pixel_order_list.append((j, colLength))
    if rowStart+1 <= rowLength:
      for k in range(colLength-1, colStart-1, -1):
        #print(matrix[rowLength][k] , " ")
        pixel_order_list.append((rowLength, k))
    if colStart+1 <= colLength:
      for k in range(rowLength-1, rowStart, -1):
        #print(matrix[k][colStart] , " ")
        pixel_order_list.append((k, colStart))

    rowStart +=1
    rowLength -=1
    colStart +=1
    colLength -=1
  # Reverse the list 
  pixel_order_list = pixel_order_list[::-1]
  for i in range(0, number_of_black_pixel):
    row = pixel_order_list[i][0]
    k = pixel_order_list[i][1]
    k+=1
    pattern[row] = pattern[row][:k-1] + [[sum(i) for i in zip([0, 0, 0], rgb_list_selected)] ] + pattern[row][k:]

  for i in range(number_of_black_pixel, len(pixel_order_list)):
    row = pixel_order_list[i][0]
    k = pixel_order_list[i][1]
    k+=1
    pattern[row] = pattern[row][:k-1] + [[sum(i) for i in zip([255, 255, 255], rgb_list_unselected_area)] ] + pattern[row][k:]

  return pattern


#https://www.geeksforgeeks.org/zigzag-or-diagonal-traversal-of-matrix/
#http://javabypatel.blogspot.com/2016/12/print-matrix-diagonally-or-diagonal.html
def diagonal_order(height, width, number_of_black_pixel):
  #img = []
  pattern = []
  pixel_order_list = []
  for y in range(height):
    #row = ()
    my_row = []
    for x in range(width):
      my_row.append('w')
      #row = row + (255, 255, 255)
    #img.append(row)
    pattern.append(my_row)

  for line in range(1, (height + width)):
    start_col = max(0, line - height)
    count = min(line, (width - start_col), height)
    for j in range(0, count):
      pixel_order_list.append((min(height, line) - j - 1, start_col + j))

  for i in range(0, number_of_black_pixel):
    r = pixel_order_list[i][0]
    k = pixel_order_list[i][1]
    k+=1
    pattern[r] = pattern[r][:k-1] + ['b'] + pattern[r][k:]
    #c = pixel_order_list[i][1]
    #c *=3
    #img[r] = img[r][:c] + (0, 0, 0) + img[r][c+3:]
  
  #print(img)
  #with open('har.png', 'wb') as f:
    #w = png.Writer(width, height, greyscale=False)
    #w.write(f, img)
  return pattern

def diagonal_order_90(height, width, number_of_black_pixel):
  #img = []
  pattern = []
  pixel_order_list = []
  for y in range(height):
    #row = ()
    my_row = []
    for x in range(width):
      my_row.append('w')
      #row = row + (255, 255, 255)
    #img.append(row)
    pattern.append(my_row)
    

  for line in range(1, (height + width)):
    start_col = max(0, line - height)
    count = min(line, (width - start_col), height)
    for j in range(0, count):
      pixel_order_list.append((min(height, line) - j - 1, start_col + j))

  for i in range(0, number_of_black_pixel):
    r = pixel_order_list[i][0]
    k = pixel_order_list[i][1]
    k+=1
    pattern[r] = pattern[r][:k-1] + ['b'] + pattern[r][k:]
    #c = pixel_order_list[i][1]
    #c *=3
    #img[r] = img[r][:c] + (0, 0, 0) + img[r][c+3:]

  '''
  a = np.asarray(img, dtype=np.int)
  a = geek.flip(a, 0)
  img = a.tolist()
  '''
  #OR
  #img = img[::-1]
  pattern = pattern[::-1]  
  #print(img)
  #with open('har.png', 'wb') as f:
    #w = png.Writer(width, height, greyscale=False)
    #w.write(f, img)
  return pattern

def diagonal_order_270(height, width, number_of_black_pixel):
  #img = []
  pattern = []
  pixel_order_list = []
  for y in range(height):
    #row = ()
    my_row = []
    for x in range(width):
      #row = row + (255, 255, 255)
      my_row.append('w')
    #img.append(row)
    pattern.append(my_row)

  for line in range(1, (height + width)):
    start_col = max(0, line - height)
    count = min(line, (width - start_col), height)
    for j in range(0, count):
      pixel_order_list.append((min(height, line) - j - 1, start_col + j))

  for i in range(0, number_of_black_pixel):
    r = pixel_order_list[i][0]
    k = pixel_order_list[i][1]
    k+=1
    pattern[r] = pattern[r][:k-1] + ['b'] + pattern[r][k:]
    #c = pixel_order_list[i][1]
    #c *=3
    #img[r] = img[r][:c] + (0, 0, 0) + img[r][c+3:]

  #a = np.asarray(img, dtype=np.int)
  #a = geek.flip(a, 1)
  #img = a.tolist()
  a = np.asarray(pattern, dtype=np.str)
  a = geek.flip(a, 1)
  pattern = a.tolist()
  
  #print(img)
  #with open('har.png', 'wb') as f:
    #w = png.Writer(width, height, greyscale=False)
    #w.write(f, img)
  return pattern

def diagonal_order_180(height, width, number_of_black_pixel):
  #img = []
  pixel_order_list = []
  pattern = []
  for y in range(height):
    #row = ()
    my_row = []
    for x in range(width):
      #row = row + (255, 255, 255)
      my_row.append('w')
    #img.append(row)
    pattern.append(my_row)

  for line in range(1, (height + width)):
    start_col = max(0, line - height)
    count = min(line, (width - start_col), height)
    for j in range(0, count):
      pixel_order_list.append((min(height, line) - j - 1, start_col + j))

  for i in range(0, number_of_black_pixel):
    r = pixel_order_list[i][0]
    #c = pixel_order_list[i][1]
    k = pixel_order_list[i][1]
    #c *=3
    k+=1
    #img[r] = img[r][:c] + (0, 0, 0) + img[r][c+3:]
    pattern[r] = pattern[r][:k-1] + ['b'] + pattern[r][k:]

  #print(img)
  #a = np.asarray(img, dtype=np.int)
  #a = geek.flip(a, 0)
  #a = geek.flip(a, 1)
  #img = a.tolist() 

  a = np.asarray(pattern, dtype=np.str)
  a = geek.flip(a, 0)
  a = geek.flip(a, 1)
  pattern = a.tolist() 
  
  #print(img)
  #with open('har.png', 'wb') as f:
    #w = png.Writer(width, height, greyscale=False)
    #w.write(f, img)
  return pattern

def my_order(height, width, number_of_black_pixel):
  # img = []
  pattern = []
  pixel_order_list = []
  for y in range(height):
    my_row = []
    for x in range(width):
      my_row.append('w')
    pattern.append(my_row)

  for r in range(0, 20, 5):
    if (r/5) % 2 :
      shift_value = 2
    else:
      shift_value = 0
    for c in range(0, 20, 5):
      for i in range(0, 3):
        for j in range(0, 3):
          pixel_order_list.append((i+r, j+c+shift_value))

  for r in range(0, 20, 5):
    if (r/5) % 2 :
      shift_value = 0
    else:
      shift_value = 3
    for c in range(0, 20, 5):
      for i in range(0, 3):
        for j in range(0, 2):
          pixel_order_list.append((i+r, j+c+shift_value))

  for r in range(0, 20, 5):
    if (r/5) % 2 :
      shift_value = 5
    else:
      shift_value = 0
    for c in range(0, 20, 10):
      for i in range(3, 5):
        for j in range(0, 5):
          pixel_order_list.append((i+r, j+c+shift_value))

  for r in range(0, 20, 5):
    if (r/5) % 2 :
      shift_value = 0
    else:
      shift_value = 5
    for c in range(0, 20, 10):
      for i in range(3, 5):
        for j in range(0, 5):
          pixel_order_list.append((i+r, j+c+shift_value))

  for i in range(0, number_of_black_pixel):
    r = pixel_order_list[i][0]
    k = pixel_order_list[i][1]
    k += 1
    pattern[r] = pattern[r][:k - 1] + ['b'] + pattern[r][k:]
  return pattern

In [65]:
list_of_matrix_walking = []
for i in range(0, segmented_data_walking.shape[0]):
  list_of_matrix_walking.append(create_matrix(data=segmented_data_walking, seg_number=i))
list_of_matrix_walking = np.array(list_of_matrix_walking)

list_of_matrix_jogging = []
for i in range(0, segmented_data_jogging.shape[0]):
  list_of_matrix_jogging.append(create_matrix(data=segmented_data_jogging, seg_number=i))
list_of_matrix_jogging = np.array(list_of_matrix_jogging)

list_of_matrix_sitting = []
for i in range(0, segmented_data_sitting.shape[0]):
  list_of_matrix_sitting.append(create_matrix(data=segmented_data_sitting, seg_number=i))
list_of_matrix_sitting = np.array(list_of_matrix_sitting)

list_of_matrix_standing = []
for i in range(0, segmented_data_standing.shape[0]):
  list_of_matrix_standing.append(create_matrix(data=segmented_data_standing, seg_number=i))
list_of_matrix_standing = np.array(list_of_matrix_standing)

list_of_matrix_upstairs = []
for i in range(0, segmented_data_upstairs.shape[0]):
  list_of_matrix_upstairs.append(create_matrix(data=segmented_data_upstairs, seg_number=i))
list_of_matrix_upstairs = np.array(list_of_matrix_upstairs)

list_of_matrix_downstairs = []
for i in range(0, segmented_data_downstairs.shape[0]):
  list_of_matrix_downstairs.append(create_matrix(data=segmented_data_downstairs, seg_number=i))
list_of_matrix_downstairs = np.array(list_of_matrix_downstairs)

In [66]:
list_of_matrix_sitting[130]

array([[2.075, 0.308, 0.842, 1.967, 0.365, 0.886],
       [1.158, 0.272, 0.376, 1.214, 0.234, 0.423],
       [0.834, 0.263, 0.401, 0.836, 0.241, 0.359],
       [4.75 , 4.48 , 4.52 , 0.88 , 0.3  , 0.5  ],
       [3.34 , 3.17 , 3.17 , 0.62 , 0.26 , 0.26 ],
       [8.16 , 8.08 , 8.2  , 0.34 , 0.19 , 0.31 ]])

In [67]:
matrix_of_stat_walking = np.zeros((3, 6, 6))
for st in range(0, 3): # index of min avg max
  for i in range(0, 6): # width of matrix 
    for j in range(0, 6): # height of matrix
      if st == 0:
        matrix_of_stat_walking[st][i][j] = min(list_of_matrix_walking[:, i, j])
      elif st == 1:
        matrix_of_stat_walking[st][i][j] = sum(list_of_matrix_walking[:, i, j])/len(list_of_matrix_walking[:, i, j])
      else:
        matrix_of_stat_walking[st][i][j] = max(list_of_matrix_walking[:, i, j])

matrix_of_stat_jogging = np.zeros((3, 6, 6))
for st in range(0, 3): # index of min avg max
  for i in range(0, 6): # width of matrix 
    for j in range(0, 6): # height of matrix
      if st == 0:
        matrix_of_stat_jogging[st][i][j] = min(list_of_matrix_jogging[:, i, j])
      elif st == 1:
        matrix_of_stat_jogging[st][i][j] = sum(list_of_matrix_jogging[:, i, j])/len(list_of_matrix_jogging[:, i, j])
      else:
        matrix_of_stat_jogging[st][i][j] = max(list_of_matrix_jogging[:, i, j])

matrix_of_stat_sitting = np.zeros((3, 6, 6))
for st in range(0, 3): # index of min avg max
  for i in range(0, 6): # width of matrix 
    for j in range(0, 6): # height of matrix
      if st == 0:
        matrix_of_stat_sitting[st][i][j] = min(list_of_matrix_sitting[:, i, j])
      elif st == 1:
        matrix_of_stat_sitting[st][i][j] = sum(list_of_matrix_sitting[:, i, j])/len(list_of_matrix_sitting[:, i, j])
      else:
        matrix_of_stat_sitting[st][i][j] = max(list_of_matrix_sitting[:, i, j])

matrix_of_stat_standing = np.zeros((3, 6, 6))
for st in range(0, 3): # index of min avg max
  for i in range(0, 6): # width of matrix 
    for j in range(0, 6): # height of matrix
      if st == 0:
        matrix_of_stat_standing[st][i][j] = min(list_of_matrix_standing[:, i, j])
      elif st == 1:
        matrix_of_stat_standing[st][i][j] = sum(list_of_matrix_standing[:, i, j])/len(list_of_matrix_standing[:, i, j])
      else:
        matrix_of_stat_standing[st][i][j] = max(list_of_matrix_standing[:, i, j])


matrix_of_stat_upstairs = np.zeros((3, 6, 6))
for st in range(0, 3): # index of min avg max
  for i in range(0, 6): # width of matrix 
    for j in range(0, 6): # height of matrix
      if st == 0:
        matrix_of_stat_upstairs[st][i][j] = min(list_of_matrix_upstairs[:, i, j])
      elif st == 1:
        matrix_of_stat_upstairs[st][i][j] = sum(list_of_matrix_upstairs[:, i, j])/len(list_of_matrix_upstairs[:, i, j])
      else:
        matrix_of_stat_upstairs[st][i][j] = max(list_of_matrix_upstairs[:, i, j])

matrix_of_stat_downstairs = np.zeros((3, 6, 6))
for st in range(0, 3): # index of min avg max
  for i in range(0, 6): # width of matrix 
    for j in range(0, 6): # height of matrix
      if st == 0:
        matrix_of_stat_downstairs[st][i][j] = min(list_of_matrix_downstairs[:, i, j])
      elif st == 1:
        matrix_of_stat_downstairs[st][i][j] = sum(list_of_matrix_downstairs[:, i, j])/len(list_of_matrix_downstairs[:, i, j])
      else:
        matrix_of_stat_downstairs[st][i][j] = max(list_of_matrix_downstairs[:, i, j])


In [68]:
matrix_of_stat_mins = np.zeros((6, 6))
matrix_of_stat_maxs = np.zeros((6, 6))
matrix_of_stat_avgs = np.zeros((6, 6))
for x in range(0, 6):
  for y in range(0, 6):
    matrix_of_stat_mins[x][y] = min(matrix_of_stat_walking[0, x, y], matrix_of_stat_jogging[0, x, y], matrix_of_stat_sitting[0, x, y],
                                    matrix_of_stat_standing[0, x, y], matrix_of_stat_downstairs[0, x, y], matrix_of_stat_upstairs[0, x, y])
    matrix_of_stat_maxs[x][y] = max([matrix_of_stat_walking[2, x, y], matrix_of_stat_jogging[2, x, y], matrix_of_stat_sitting[2, x, y],
                                    matrix_of_stat_standing[2, x, y], matrix_of_stat_downstairs[2, x, y], matrix_of_stat_upstairs[2, x, y]])
    matrix_of_stat_avgs[x][y] = round(sum([matrix_of_stat_walking[1, x, y], matrix_of_stat_jogging[1, x, y], matrix_of_stat_sitting[1, x, y],
                                    matrix_of_stat_standing[1, x, y], matrix_of_stat_downstairs[1, x, y], matrix_of_stat_upstairs[1, x, y]])/6, 2)

In [69]:
# TODO Better to name it conversion mechanism or function
def cal_val_in_area_distriution(number, min_interval_value, max_interval_value):
  result = round(((w_area*h_area)/(max_interval_value - min_interval_value))*(number-min_interval_value), 0)
  return int(result)

def cal_val_in_rgb_distriution(number, min_interval_value, max_interval_value):
  result = round((255/(max_interval_value - min_interval_value))*(number-min_interval_value), 0)
  return int(result)



In [70]:
def get_patterns(area_height, area_width, pre_matrix, curr_matrix, post_matrix, pre_2_matrix, post_2_matrix):

  matrix_of_image_values=np.zeros((6, 6))
  matrix_of_image_values_pre=np.zeros((6, 6))
  matrix_of_image_values_curr=np.zeros((6, 6))
  matrix_of_image_values_post=np.zeros((6, 6))

  matrix_of_image_values_pre_2=np.zeros((6, 6))
  matrix_of_image_values_post_2=np.zeros((6, 6))

  matrix_of_pattern = []
  counter = 0
  for i in range(0, 6):
    for j in range(0, 6):
      matrix_of_image_values[i][j] = cal_val_in_area_distriution((curr_matrix[i, j]), matrix_of_stat_mins[i, j], matrix_of_stat_maxs[i, j])
      matrix_of_image_values_pre[i][j] = cal_val_in_rgb_distriution((pre_matrix[i, j]), matrix_of_stat_mins[i, j], matrix_of_stat_maxs[i, j])
      matrix_of_image_values_curr[i][j] = cal_val_in_rgb_distriution((curr_matrix[i, j]), matrix_of_stat_mins[i, j], matrix_of_stat_maxs[i, j])
      matrix_of_image_values_post[i][j] = cal_val_in_rgb_distriution((post_matrix[i, j]), matrix_of_stat_mins[i, j], matrix_of_stat_maxs[i, j])

      matrix_of_image_values_pre_2[i][j] = cal_val_in_rgb_distriution((pre_2_matrix[i, j]), matrix_of_stat_mins[i, j], matrix_of_stat_maxs[i, j])
      matrix_of_image_values_post_2[i][j] = cal_val_in_rgb_distriution((post_2_matrix[i, j]), matrix_of_stat_mins[i, j], matrix_of_stat_maxs[i, j])

  matrix_of_image_values=matrix_of_image_values.astype(int)
  matrix_of_image_values_pre=matrix_of_image_values_pre.astype(int)
  matrix_of_image_values_curr=matrix_of_image_values_curr.astype(int)
  matrix_of_image_values_post=matrix_of_image_values_post.astype(int)

  matrix_of_image_values_pre_2=matrix_of_image_values_pre_2.astype(int)
  matrix_of_image_values_post_2=matrix_of_image_values_post_2.astype(int)

  #### Patern calculation
  pattern0 = counter_clockwise_outwards(area_width, area_height, matrix_of_image_values[0, 0], 
                                        [matrix_of_image_values_pre[0, 0], matrix_of_image_values_curr[0, 0], matrix_of_image_values_post[0, 0]], 
                                        [matrix_of_image_values_pre_2[0, 0], matrix_of_image_values_curr[0, 0], matrix_of_image_values_post_2[0, 0]])
  pattern1 = clockwise_inwards(area_width, area_height, matrix_of_image_values[0, 1], 
                               [matrix_of_image_values_pre[0, 1], matrix_of_image_values_curr[0, 1], matrix_of_image_values_post[0, 1]], 
                               [matrix_of_image_values_pre_2[0, 1], matrix_of_image_values_curr[0, 1], matrix_of_image_values_post_2[0, 1]])
  pattern2 = counter_clockwise_outwards(area_width, area_height, matrix_of_image_values[0, 2], 
                                        [matrix_of_image_values_pre[0, 2], matrix_of_image_values_curr[0, 2], matrix_of_image_values_post[0, 2]], 
                                        [matrix_of_image_values_pre_2[0, 2], matrix_of_image_values_curr[0, 2], matrix_of_image_values_post_2[0, 2]])
  pattern3 = clockwise_inwards(area_width, area_height, matrix_of_image_values[0, 3], 
                               [matrix_of_image_values_pre[0, 3], matrix_of_image_values_curr[0, 3], matrix_of_image_values_post[0, 3]], 
                               [matrix_of_image_values_pre_2[0, 3], matrix_of_image_values_curr[0, 3], matrix_of_image_values_post_2[0, 3]])
  pattern4 = counter_clockwise_outwards(area_width, area_height, matrix_of_image_values[0, 4], 
                                        [matrix_of_image_values_pre[0, 4], matrix_of_image_values_curr[0, 4], matrix_of_image_values_post[0, 4]], 
                                        [matrix_of_image_values_pre_2[0, 4], matrix_of_image_values_curr[0, 4], matrix_of_image_values_post_2[0, 4]])
  pattern5 = clockwise_inwards(area_width, area_height, matrix_of_image_values[0, 5], 
                               [matrix_of_image_values_pre[0, 5], matrix_of_image_values_curr[0, 5], matrix_of_image_values_post[0, 5]], 
                               [matrix_of_image_values_pre_2[0, 5], matrix_of_image_values_curr[0, 5], matrix_of_image_values_post_2[0, 5]])
  pattern6 = clockwise_inwards(area_width, area_height, matrix_of_image_values[1, 0], 
                               [matrix_of_image_values_pre[1, 0], matrix_of_image_values_curr[1, 0], matrix_of_image_values_post[1, 0]], 
                               [matrix_of_image_values_pre_2[1, 0], matrix_of_image_values_curr[1, 0], matrix_of_image_values_post_2[1, 0]])
  pattern7 = counter_clockwise_outwards(area_width, area_height, matrix_of_image_values[1, 1], 
                                        [matrix_of_image_values_pre[1, 1], matrix_of_image_values_curr[1, 1], matrix_of_image_values_post[1, 1]], 
                                        [matrix_of_image_values_pre_2[1, 1], matrix_of_image_values_curr[1, 1], matrix_of_image_values_post_2[1, 1]])
  pattern8 = clockwise_inwards(area_width, area_height, matrix_of_image_values[1, 2], 
                               [matrix_of_image_values_pre[1, 2], matrix_of_image_values_curr[1, 2], matrix_of_image_values_post[1, 2]], 
                               [matrix_of_image_values_pre_2[1, 2], matrix_of_image_values_curr[1, 2], matrix_of_image_values_post_2[1, 2]])
  pattern9 = counter_clockwise_outwards(area_width, area_height, matrix_of_image_values[1, 3], 
                                        [matrix_of_image_values_pre[1, 3], matrix_of_image_values_curr[1, 3], matrix_of_image_values_post[1, 3]], 
                                        [matrix_of_image_values_pre_2[1, 3], matrix_of_image_values_curr[1, 3], matrix_of_image_values_post_2[1, 3]])
  pattern10 = clockwise_inwards(area_width, area_height, matrix_of_image_values[1, 4], 
                                [matrix_of_image_values_pre[1, 4], matrix_of_image_values_curr[1, 4], matrix_of_image_values_post[1, 4]], 
                                [matrix_of_image_values_pre_2[1, 4], matrix_of_image_values_curr[1, 4], matrix_of_image_values_post_2[1, 4]])
  pattern11 = counter_clockwise_outwards(area_width, area_height, matrix_of_image_values[1, 5], 
                                         [matrix_of_image_values_pre[1, 5], matrix_of_image_values_curr[1, 5], matrix_of_image_values_post[1, 5]], 
                                         [matrix_of_image_values_pre_2[1, 5], matrix_of_image_values_curr[1, 5], matrix_of_image_values_post_2[1, 5]])
  pattern12 = counter_clockwise_outwards(area_width, area_height, matrix_of_image_values[2, 0], 
                                         [matrix_of_image_values_pre[2, 0], matrix_of_image_values_curr[2, 0], matrix_of_image_values_post[2, 0]], 
                                         [matrix_of_image_values_pre_2[2, 0], matrix_of_image_values_curr[2, 0], matrix_of_image_values_post_2[2, 0]])
  pattern13 = clockwise_inwards(area_width, area_height, matrix_of_image_values[2, 1], 
                                [matrix_of_image_values_pre[2, 1], matrix_of_image_values_curr[2, 1], matrix_of_image_values_post[2, 1]], 
                                [matrix_of_image_values_pre_2[2, 1], matrix_of_image_values_curr[2, 1], matrix_of_image_values_post_2[2, 1]])
  pattern14 = counter_clockwise_outwards(area_width, area_height, matrix_of_image_values[2, 2], 
                                         [matrix_of_image_values_pre[2, 2], matrix_of_image_values_curr[2, 2], matrix_of_image_values_post[2, 2]], 
                                         [matrix_of_image_values_pre_2[2, 2], matrix_of_image_values_curr[2, 2], matrix_of_image_values_post_2[2, 2]])
  pattern15 = clockwise_inwards(area_width, area_height, matrix_of_image_values[2, 3], 
                                [matrix_of_image_values_pre[2, 3], matrix_of_image_values_curr[2, 3], matrix_of_image_values_post[2, 3]], 
                                [matrix_of_image_values_pre_2[2, 3], matrix_of_image_values_curr[2, 3], matrix_of_image_values_post_2[2, 3]])
  pattern16 = counter_clockwise_outwards(area_width, area_height, matrix_of_image_values[2, 4], 
                                         [matrix_of_image_values_pre[2, 4], matrix_of_image_values_curr[2, 4], matrix_of_image_values_post[2, 4]], 
                                         [matrix_of_image_values_pre_2[2, 4], matrix_of_image_values_curr[2, 4], matrix_of_image_values_post_2[2, 4]])
  pattern17 = clockwise_inwards(area_width, area_height, matrix_of_image_values[2, 5], 
                                [matrix_of_image_values_pre[2, 5], matrix_of_image_values_curr[2, 5], matrix_of_image_values_post[2, 5]], 
                                [matrix_of_image_values_pre_2[2, 5], matrix_of_image_values_curr[2, 5], matrix_of_image_values_post_2[2, 5]])

  pattern18 = clockwise_inwards(area_width, area_height, matrix_of_image_values[3, 0], 
                                [matrix_of_image_values_pre[3, 0], matrix_of_image_values_curr[3, 0], matrix_of_image_values_post[3, 0]], 
                                [matrix_of_image_values_pre_2[3, 0], matrix_of_image_values_curr[3, 0], matrix_of_image_values_post_2[3, 0]])
  pattern19 = counter_clockwise_outwards(area_width, area_height, matrix_of_image_values[3, 1], 
                                         [matrix_of_image_values_pre[3, 1], matrix_of_image_values_curr[3, 1], matrix_of_image_values_post[3, 1]], 
                                         [matrix_of_image_values_pre_2[3, 1], matrix_of_image_values_curr[3, 1], matrix_of_image_values_post_2[3, 1]])
  pattern20 = clockwise_inwards(area_width, area_height, matrix_of_image_values[3, 2], 
                                [matrix_of_image_values_pre[3, 2], matrix_of_image_values_curr[3, 2], matrix_of_image_values_post[3, 2]], 
                                [matrix_of_image_values_pre_2[3, 2], matrix_of_image_values_curr[3, 2], matrix_of_image_values_post_2[3, 2]])
  pattern21 = counter_clockwise_outwards(area_width, area_height, matrix_of_image_values[3, 3], 
                                         [matrix_of_image_values_pre[3, 3], matrix_of_image_values_curr[3, 3], matrix_of_image_values_post[3, 3]], 
                                         [matrix_of_image_values_pre_2[3, 3], matrix_of_image_values_curr[3, 3], matrix_of_image_values_post_2[3, 3]])
  pattern22 = clockwise_inwards(area_width, area_height, matrix_of_image_values[3, 4], 
                                [matrix_of_image_values_pre[3, 4], matrix_of_image_values_curr[3, 4], matrix_of_image_values_post[3, 4]], 
                                [matrix_of_image_values_pre_2[3, 4], matrix_of_image_values_curr[3, 4], matrix_of_image_values_post_2[3, 4]])
  pattern23 = counter_clockwise_outwards(area_width, area_height, matrix_of_image_values[3, 5],
                                         [matrix_of_image_values_pre[3, 5], matrix_of_image_values_curr[3, 5], matrix_of_image_values_post[3, 5]], 
                                         [matrix_of_image_values_pre_2[3, 5], matrix_of_image_values_curr[3, 5], matrix_of_image_values_post_2[3, 5]])
  pattern24 = counter_clockwise_outwards(area_width, area_height, matrix_of_image_values[4, 0], 
                                         [matrix_of_image_values_pre[4, 0], matrix_of_image_values_curr[4, 0], matrix_of_image_values_post[4, 0]], 
                                         [matrix_of_image_values_pre_2[4, 0], matrix_of_image_values_curr[4, 0], matrix_of_image_values_post_2[4, 0]])
  pattern25 = clockwise_inwards(area_width, area_height, matrix_of_image_values[4, 1], 
                                [matrix_of_image_values_pre[4, 1], matrix_of_image_values_curr[4, 1], matrix_of_image_values_post[4, 1]], 
                                [matrix_of_image_values_pre_2[4, 1], matrix_of_image_values_curr[4, 1], matrix_of_image_values_post_2[4, 1]])
  pattern26 = counter_clockwise_outwards(area_width, area_height, matrix_of_image_values[4, 2], 
                                         [matrix_of_image_values_pre[4, 2], matrix_of_image_values_curr[4, 2], matrix_of_image_values_post[4, 2]], 
                                         [matrix_of_image_values_pre_2[4, 2], matrix_of_image_values_curr[4, 2], matrix_of_image_values_post_2[4, 2]])
  pattern27 = clockwise_inwards(area_width, area_height, matrix_of_image_values[4, 3], 
                                [matrix_of_image_values_pre[4, 3], matrix_of_image_values_curr[4, 3], matrix_of_image_values_post[4, 3]], 
                                [matrix_of_image_values_pre_2[4, 3], matrix_of_image_values_curr[4, 3], matrix_of_image_values_post_2[4, 3]])
  pattern28 = counter_clockwise_outwards(area_width, area_height, matrix_of_image_values[4, 4], 
                                         [matrix_of_image_values_pre[4, 4], matrix_of_image_values_curr[4, 4], matrix_of_image_values_post[4, 4]], 
                                         [matrix_of_image_values_pre_2[4, 4], matrix_of_image_values_curr[4, 4], matrix_of_image_values_post_2[4, 4]])
  pattern29 = clockwise_inwards(area_width, area_height, matrix_of_image_values[4, 5], 
                                [matrix_of_image_values_pre[4, 5], matrix_of_image_values_curr[4, 5], matrix_of_image_values_post[4, 5]], 
                                [matrix_of_image_values_pre_2[4, 5], matrix_of_image_values_curr[4, 5], matrix_of_image_values_post_2[4, 5]])
  pattern30 = clockwise_inwards(area_width, area_height, matrix_of_image_values[5, 0], 
                                [matrix_of_image_values_pre[5, 0], matrix_of_image_values_curr[5, 0], matrix_of_image_values_post[5, 0]], 
                                [matrix_of_image_values_pre_2[5, 0], matrix_of_image_values_curr[5, 0], matrix_of_image_values_post_2[5, 0]]) 
  pattern31 = counter_clockwise_outwards(area_width, area_height, matrix_of_image_values[5, 1], 
                                         [matrix_of_image_values_pre[5, 1], matrix_of_image_values_curr[5, 1], matrix_of_image_values_post[5, 1]], 
                                         [matrix_of_image_values_pre_2[5, 1], matrix_of_image_values_curr[5, 1], matrix_of_image_values_post_2[5, 1]])
  pattern32 = clockwise_inwards(area_width, area_height, matrix_of_image_values[5, 2], 
                                [matrix_of_image_values_pre[5, 2], matrix_of_image_values_curr[5, 2], matrix_of_image_values_post[5, 2]], 
                                [matrix_of_image_values_pre_2[5, 2], matrix_of_image_values_curr[5, 2], matrix_of_image_values_post_2[5, 2]])
  pattern33 = counter_clockwise_outwards(area_width, area_height, matrix_of_image_values[5, 3], 
                                         [matrix_of_image_values_pre[5, 3], matrix_of_image_values_curr[5, 3], matrix_of_image_values_post[5, 3]], 
                                         [matrix_of_image_values_pre_2[5, 3], matrix_of_image_values_curr[5, 3], matrix_of_image_values_post_2[5, 3]])
  pattern34 = clockwise_inwards(area_width, area_height, matrix_of_image_values[5, 4], 
                                [matrix_of_image_values_pre[5, 4], matrix_of_image_values_curr[5, 4], matrix_of_image_values_post[5, 4]], 
                                [matrix_of_image_values_pre_2[5, 4], matrix_of_image_values_curr[5, 4], matrix_of_image_values_post_2[5, 4]])
  pattern35 = counter_clockwise_outwards(area_width, area_height, matrix_of_image_values[5, 5], 
                                         [matrix_of_image_values_pre[5, 5], matrix_of_image_values_curr[5, 5], matrix_of_image_values_post[5, 5]], 
                                         [matrix_of_image_values_pre_2[5, 5], matrix_of_image_values_curr[5, 5], matrix_of_image_values_post_2[5, 5]])
  
  for elm in pattern1:
    pattern0.append(elm)
  for elm in pattern2:
    pattern0.append(elm)
  for elm in pattern3:
    pattern0.append(elm)
  for elm in pattern4:
    pattern0.append(elm)
  for elm in pattern5:
    pattern0.append(elm)
  for elm in pattern6:
    pattern0.append(elm)
  for elm in pattern7:
    pattern0.append(elm)
  for elm in pattern8:
    pattern0.append(elm)
  for elm in pattern9:
    pattern0.append(elm)
  for elm in pattern10:
    pattern0.append(elm)
  for elm in pattern11:
    pattern0.append(elm)
  for elm in pattern12:
    pattern0.append(elm)
  for elm in pattern13:
    pattern0.append(elm)
  for elm in pattern14:
    pattern0.append(elm)
  for elm in pattern15:
    pattern0.append(elm)
  for elm in pattern16:
    pattern0.append(elm)
  for elm in pattern17:
    pattern0.append(elm)
  for elm in pattern18:
    pattern0.append(elm)
  for elm in pattern19:
    pattern0.append(elm)
  for elm in pattern20:
    pattern0.append(elm)
  for elm in pattern21:
    pattern0.append(elm)
  for elm in pattern22:
    pattern0.append(elm)
  for elm in pattern23:
    pattern0.append(elm)
  for elm in pattern24:
    pattern0.append(elm)
  for elm in pattern25:
    pattern0.append(elm)
  for elm in pattern26:
    pattern0.append(elm)
  for elm in pattern27:
    pattern0.append(elm)
  for elm in pattern28:
    pattern0.append(elm)
  for elm in pattern29:
    pattern0.append(elm)
  for elm in pattern30:
    pattern0.append(elm)
  for elm in pattern31:
    pattern0.append(elm)
  for elm in pattern32:
    pattern0.append(elm)
  for elm in pattern33:
    pattern0.append(elm)
  for elm in pattern34:
    pattern0.append(elm)
  for elm in pattern35:
    pattern0.append(elm) 
            
  return pattern0        

def create_img(area_height, area_width, number_of_area_x, number_of_area_y, 
               pre_matrix, matrix, post_matrix, 
               pre_2_matrix, post_2_matrix, 
               name_of_file):
  pattern1 = get_patterns(area_height, area_width, pre_matrix, matrix, post_matrix, pre_2_matrix, post_2_matrix)
  height = area_height * number_of_area_y
  width = area_width * number_of_area_x
  matrix_of_var = [[[255, 255, 255] for x in range(width)] for y in range(height)]
  p =-1  
  temp = np.array(pattern1)
  #print(temp.shape)
  for i in range(0, number_of_area_y):
    k = i*area_height
    for j in range(0, number_of_area_x):
      z = j * area_width
      for x in range(k, k + area_width):
        if p < (area_width*(number_of_area_x * number_of_area_y))-1:
          p+=1
        q=-1
        for y in range(z, z + area_height):
          q+=1
          #print("({}, {})".format(x, y))
          #print("({}, {})".format(p, q))
          matrix_of_var[x][y] = pattern1[p][q]
          #print("p={} and 1={}".format(p, q))
  #print(matrix_of_var.flatten()
  #print(matrix_of_var)
  #print(pattern1)
  for idx in range(len(matrix_of_var)):
    for i in range(len(matrix_of_var[idx])):
        matrix_of_var[idx][i] = tuple(matrix_of_var[idx][i])
  a = np.asarray(matrix_of_var, dtype=np.int)
  a = a.reshape(a.shape[0], a.shape[1]*a.shape[2])
  a = a.tolist()
  with open(name_of_file, 'wb') as f:
    w = png.Writer(width, height, greyscale=False)
    w.write(f, a)
#diagonal_order_180(20, 20, 200)
                  

In [74]:
import os, shutil
from pathlib import Path
folder = '/Users/mesrbn/Desktop/Data_Gen_Codes/WISDM/Gen_Data'
Path(folder).mkdir(parents=True, exist_ok=True)
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))
Path(folder+'/Walking').mkdir(parents=True, exist_ok=True)
Path(folder+'/Standing').mkdir(parents=True, exist_ok=True)
Path(folder+'/Upstairs').mkdir(parents=True, exist_ok=True)
Path(folder+'/Jogging').mkdir(parents=True, exist_ok=True)
Path(folder+'/Downstairs').mkdir(parents=True, exist_ok=True)
Path(folder+'/Sitting').mkdir(parents=True, exist_ok=True)

In [76]:
import math
index = 1
num = len(list_of_matrix_sitting)
for i in range(0, num):
  name_of_file = '/Users/mesrbn/Desktop/Data_Gen_Codes/WISDM/Gen_Data/Sitting/Sitting_' +  str(index) + '.png'
  index +=1
  if i == 0:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               matrix_of_stat_sitting[0, :, :], list_of_matrix_sitting[i, :, :], list_of_matrix_sitting[i+1, :, :], 
               matrix_of_stat_sitting[0, :, :], list_of_matrix_sitting[i+2, :, :], 
               name_of_file)
  elif i == num-1:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_sitting[i-1, :, :], list_of_matrix_sitting[i, :, :], matrix_of_stat_sitting[0, :, :], 
               list_of_matrix_sitting[i-2, :, :], matrix_of_stat_sitting[0, :, :],
               name_of_file)
  elif i == 1:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_sitting[i-1, :, :], list_of_matrix_sitting[i, :, :], list_of_matrix_sitting[i+1, :, :], 
               matrix_of_stat_sitting[0, :, :], list_of_matrix_sitting[i+2, :, :], 
               name_of_file)
  elif i == num-2:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_sitting[i-1, :, :], list_of_matrix_sitting[i, :, :], list_of_matrix_sitting[i+1, :, :], 
               list_of_matrix_sitting[i-2, :, :], matrix_of_stat_sitting[0, :, :],
               name_of_file)  
  else:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_sitting[i-1, :, :], list_of_matrix_sitting[i, :, :], list_of_matrix_sitting[i+1, :, :], 
               list_of_matrix_sitting[i-2, :, :], list_of_matrix_sitting[i+2, :, :], 
               name_of_file)

/var/folders/w3/skl35jlj5jzc2cvk9z8nxw640000gn/T/ipykernel_90438/2487312622.py:246: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  a = np.asarray(matrix_of_var, dtype=np.int)


In [77]:
import math
index = 1
num = len(list_of_matrix_standing)
for i in range(0, num):
  name_of_file = '/content/Gen_Data/Standing/Standing_' +  str(index) + '.png'
  index +=1
  if i == 0:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               matrix_of_stat_standing[0, :, :], list_of_matrix_standing[i, :, :], list_of_matrix_standing[i+1, :, :], 
               matrix_of_stat_standing[0, :, :], list_of_matrix_standing[i+2, :, :], 
               name_of_file)
  elif i == num-1:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_standing[i-1, :, :], list_of_matrix_standing[i, :, :], matrix_of_stat_standing[0, :, :], 
               list_of_matrix_standing[i-2, :, :], matrix_of_stat_standing[0, :, :],
               name_of_file)
  elif i == 1:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_standing[i-1, :, :], list_of_matrix_standing[i, :, :], list_of_matrix_standing[i+1, :, :], 
               matrix_of_stat_standing[0, :, :], list_of_matrix_standing[i+2, :, :], 
               name_of_file)
  elif i == num-2:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_standing[i-1, :, :], list_of_matrix_standing[i, :, :], list_of_matrix_standing[i+1, :, :], 
               list_of_matrix_standing[i-2, :, :], matrix_of_stat_standing[0, :, :],
               name_of_file)  
  else:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_standing[i-1, :, :], list_of_matrix_standing[i, :, :], list_of_matrix_standing[i+1, :, :], 
               list_of_matrix_standing[i-2, :, :], list_of_matrix_standing[i+2, :, :], 
               name_of_file)

/var/folders/w3/skl35jlj5jzc2cvk9z8nxw640000gn/T/ipykernel_90438/2487312622.py:246: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  a = np.asarray(matrix_of_var, dtype=np.int)


FileNotFoundError: [Errno 2] No such file or directory: '/content/Gen_Data/Standing/Standing_1.png'

In [ ]:
import math
index = 1
num = len(list_of_matrix_walking)
for i in range(0, num):
  name_of_file = '/content/Gen_Data/Walking/Walking_' +  str(index) + '.png'
  index +=1
  if i == 0:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               matrix_of_stat_walking[0, :, :], list_of_matrix_walking[i, :, :], list_of_matrix_walking[i+1, :, :], 
               matrix_of_stat_walking[0, :, :], list_of_matrix_walking[i+2, :, :], 
               name_of_file)
  elif i == num-1:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_walking[i-1, :, :], list_of_matrix_walking[i, :, :], matrix_of_stat_walking[0, :, :], 
               list_of_matrix_walking[i-2, :, :], matrix_of_stat_walking[0, :, :],
               name_of_file)
  elif i == 1:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_walking[i-1, :, :], list_of_matrix_walking[i, :, :], list_of_matrix_walking[i+1, :, :], 
               matrix_of_stat_walking[0, :, :], list_of_matrix_walking[i+2, :, :], 
               name_of_file)
  elif i == num-2:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_walking[i-1, :, :], list_of_matrix_walking[i, :, :], list_of_matrix_walking[i+1, :, :], 
               list_of_matrix_walking[i-2, :, :], matrix_of_stat_walking[0, :, :],
               name_of_file)  
  else:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_walking[i-1, :, :], list_of_matrix_walking[i, :, :], list_of_matrix_walking[i+1, :, :], 
               list_of_matrix_walking[i-2, :, :], list_of_matrix_walking[i+2, :, :], 
               name_of_file)

In [ ]:
import math
index = 1
num = len(list_of_matrix_jogging)
#print(list_of_matrix_walking[0, :, :])
for i in range(0, num):
  name_of_file = '/content/Gen_Data/Jogging/Jogging_' +  str(index) + '.png'
  #name_of_file = '/content/drive/My Drive/Thesis-Data/W200/Jogging/Jogging_' +  str(index) + '.png'
  index +=1
  if i == 0:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               matrix_of_stat_jogging[0, :, :], list_of_matrix_jogging[i, :, :], list_of_matrix_jogging[i+1, :, :], 
               matrix_of_stat_jogging[0, :, :], list_of_matrix_jogging[i+2, :, :], 
               name_of_file)
  elif i == num-1:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_jogging[i-1, :, :], list_of_matrix_jogging[i, :, :], matrix_of_stat_jogging[0, :, :], 
               list_of_matrix_jogging[i-2, :, :], matrix_of_stat_jogging[0, :, :],
               name_of_file)
  elif i == 1:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_jogging[i-1, :, :], list_of_matrix_jogging[i, :, :], list_of_matrix_jogging[i+1, :, :], 
               matrix_of_stat_jogging[0, :, :], list_of_matrix_jogging[i+2, :, :], 
               name_of_file)
  elif i == num-2:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_jogging[i-1, :, :], list_of_matrix_jogging[i, :, :], list_of_matrix_jogging[i+1, :, :], 
               list_of_matrix_jogging[i-2, :, :], matrix_of_stat_jogging[0, :, :],
               name_of_file)  
  else:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_jogging[i-1, :, :], list_of_matrix_jogging[i, :, :], list_of_matrix_jogging[i+1, :, :], 
               list_of_matrix_jogging[i-2, :, :], list_of_matrix_jogging[i+2, :, :], 
               name_of_file)

In [ ]:
import math
index = 1
num = len(list_of_matrix_upstairs)
#print(list_of_matrix_walking[0, :, :])
for i in range(0, num):
  name_of_file = '/content/Gen_Data/Upstairs/Upstairs_' +  str(index) + '.png'
  #name_of_file = '/content/drive/My Drive/Thesis-Data/W200/Upstairs/Upstairs_' +  str(index) + '.png'
  index +=1
  if i == 0:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               matrix_of_stat_upstairs[0, :, :], list_of_matrix_upstairs[i, :, :], list_of_matrix_upstairs[i+1, :, :], 
               matrix_of_stat_upstairs[0, :, :], list_of_matrix_upstairs[i+2, :, :], 
               name_of_file)
  elif i == num-1:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_upstairs[i-1, :, :], list_of_matrix_upstairs[i, :, :], matrix_of_stat_upstairs[0, :, :], 
               list_of_matrix_upstairs[i-2, :, :], matrix_of_stat_upstairs[0, :, :],
               name_of_file)
  elif i == 1:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_upstairs[i-1, :, :], list_of_matrix_upstairs[i, :, :], list_of_matrix_upstairs[i+1, :, :], 
               matrix_of_stat_upstairs[0, :, :], list_of_matrix_upstairs[i+2, :, :], 
               name_of_file)
  elif i == num-2:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_upstairs[i-1, :, :], list_of_matrix_upstairs[i, :, :], list_of_matrix_upstairs[i+1, :, :], 
               list_of_matrix_upstairs[i-2, :, :], matrix_of_stat_upstairs[0, :, :],
               name_of_file)  
  else:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_upstairs[i-1, :, :], list_of_matrix_upstairs[i, :, :], list_of_matrix_upstairs[i+1, :, :], 
               list_of_matrix_upstairs[i-2, :, :], list_of_matrix_upstairs[i+2, :, :], 
               name_of_file)

In [ ]:
import math
index = 1
num = len(list_of_matrix_downstairs)
#print(list_of_matrix_walking[0, :, :])
for i in range(0, num):
  name_of_file = '/content/Gen_Data/Downstairs/Downstairs_' +  str(index) + '.png'
  #name_of_file = '/content/drive/My Drive/Thesis-Data/W200/Downstairs/Downstairs_' +  str(index) + '.png'
  index +=1
  if i == 0:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               matrix_of_stat_downstairs[0, :, :], list_of_matrix_downstairs[i, :, :], list_of_matrix_downstairs[i+1, :, :], 
               matrix_of_stat_downstairs[0, :, :], list_of_matrix_downstairs[i+2, :, :], 
               name_of_file)
  elif i == num-1:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_downstairs[i-1, :, :], list_of_matrix_downstairs[i, :, :], matrix_of_stat_downstairs[0, :, :], 
               list_of_matrix_downstairs[i-2, :, :], matrix_of_stat_downstairs[0, :, :],
               name_of_file)
  elif i == 1:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_downstairs[i-1, :, :], list_of_matrix_downstairs[i, :, :], list_of_matrix_downstairs[i+1, :, :], 
               matrix_of_stat_downstairs[0, :, :], list_of_matrix_downstairs[i+2, :, :], 
               name_of_file)
  elif i == num-2:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_downstairs[i-1, :, :], list_of_matrix_downstairs[i, :, :], list_of_matrix_downstairs[i+1, :, :], 
               list_of_matrix_downstairs[i-2, :, :], matrix_of_stat_downstairs[0, :, :],
               name_of_file)  
  else:
    create_img(w_area, h_area, num_area_w, num_area_h, 
               list_of_matrix_downstairs[i-1, :, :], list_of_matrix_downstairs[i, :, :], list_of_matrix_downstairs[i+1, :, :], 
               list_of_matrix_downstairs[i-2, :, :], list_of_matrix_downstairs[i+2, :, :], 
               name_of_file)

In [ ]:
import shutil
file_name = name = 'my_data_W' + str(MY_WINDOW_SIZE) + "_" + str(w_area*num_area_w) + '*' +  str(h_area*num_area_h)
shutil.make_archive(file_name, 'zip', '/content/Gen_Data')